# Chapter 1: Characterizing Seasonal Rainfall Variability

<div class="alert alert-info">
 [⬅️ Previous Chapter](Chapter 0.ipynb)     [➡️ Next Chapter](Chapter 2.ipynb)
</div>

Rainfall in many dryland ecosystems exhibits pronounced variability from year to year. In this excercise, we will investigate the nature of rainfall at some locations in Kenya and Zambia and use these data to draw inference into the nature of the rainfall process. Finally, we will use the data to develop an empirical model of rainfall that we could use to simulate the rainfall process across many realizations of discrete growing seasons. 

## Chapter Objectives

* Analyze daily rainfall at monthly, seasonal, and yearly scales
* Examine the nature of the daily rainfall process
* Plot key results and compare findings across stations and over time.

First, let's load pandas into our notebook. We use `pd` as the shortname for pandas, which is an informal convention that you will see used across the python data analysis landscape. 

<div class="alert alert-warning">
**NOTE: You only need to run the next cell once per session.**
</div>

In [97]:
import pandas as pd

## Reading in the data

All the data for our excercise are stored as `.csv` files. These files are very easy to import into pandas. We read the `.csv` file into a `dataframe`, which is the fundamental unit of storage and analysis pandas. The `dataframe` is similar to `R` dataframes, and can even be thought of as a spreadsheet. Every dataframe contains columns (variables) and rows (values).

In [98]:
df = pd.read_csv("../data/mpala_ranch_house_rainfall.csv")

#### What just happened: 
Our `.csv` file was just imported by pandas and stored in a pandas dataframe, which we called `df` (another common shorthand in the pandas world). The resulting `df` object will allow us to do all sorts of analysis and manipulations of the data, just as if we had loaded the data into a spreadsheet in Excel. 

If you're not familiar, this would be a good time to check out a pandas tutorial, which is available in [Chapter 0](Chapter 0.ipynb) of this module.

<div class="alert alert-info">
The **info()** command provides a quick summary of our dataframe's contents.
</div>

In [99]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16602 entries, 0 to 16601
Data columns (total 5 columns):
Rainfall (mm)    16602 non-null float64
Date             16602 non-null object
Year             16602 non-null int64
Month            16601 non-null float64
Day              16601 non-null float64
dtypes: float64(3), int64(1), object(1)
memory usage: 648.6+ KB


Here we see that the dataframe has 16,602 entries (this is the number of rows in the `.csv` file), and contains 5 columns that are called `Rainfall (mm)`, `Date`, `Year`, `Month`, and `Day`. These column names are taken directly from the `.csv` header row.

Pandas has auto-detected that `Year` is an integer and the data in the `Rainfall (mm)` column are detected to be floating point numbers. However, `Month` and `Day`, which should also be integers have been classified as `float64`, while the `Date` column has been categorized as an indeterminant `object` type (this is the default type, which means pandas has no idea what to do with this column). We will come back to this in a bit, but first let's keep exploring our new dataframe.


<div class="alert alert-info">Let's take a look at the first few rows of the dataframe using the **head()** command.</div>

In [100]:
df.head()

,Rainfall (mm),Date,Year,Month,Day
0,0.0,1/1/72,1972,1.0,1.0
1,0.0,1/2/72,1972,1.0,2.0
2,0.0,1/3/72,1972,1.0,3.0
3,0.0,1/4/72,1972,1.0,4.0
4,0.0,1/5/72,1972,1.0,5.0


We see the four columns of data we expect to see, plus an additional _Index column_ that is zero-indexed on the far left. This index column is automatically added by pandaes to every dataframe. Later on, we will see how to specify the index column contents.

The **info()** command reports the columns and datatypes in a dataframe, and the **head()** command provides the first few rows of values. 

<div class="alert alert-info">The **describe()** command provides a simple statistical summary of our dataframe.</div>

In [101]:
df.describe()

,Rainfall (mm),Year,Month,Day
count,16602.000000,16602.000000,16601.000000,16601.000000
mean,-511.036106,1994.227322,6.490633,15.721523
std,2205.461049,13.125253,3.450448,8.800142
min,-9999.000000,1972.000000,1.000000,1.000000
25%,0.000000,1983.000000,3.000000,8.000000
50%,0.000000,1994.000000,6.000000,16.000000
75%,0.000000,2006.000000,9.000000,23.000000
max,635.000000,2017.000000,12.000000,31.000000


The **describe()** function has provided summaries of each of our columns. We see that something is amiss in the `Rainfall (mm)` column; the minimum value is `-9999` and the mean value is less than zero! It looks like we have some things to sort out before we proceed.

### Cleaning up our data

If building a boat is a "bit of construction followed by a ton of sanding", then conducting data analysis is usually a "ton of cleaning followed by a bit of analysis". While it's tempting to hide this process from you during this module, it's helpful to learn a little about how to clean up data. 

The first thing we want to do is reassign the `-9999` values of rainfall to `nan`, which means "not a number", which will cause `pandas` to ignore them in all of its calculations.

Next, we want to clean up the column types so they correspond to what we know them to be. First, let's deal with the `Day` and `Month` columns. Let's use the **unique()** function to inspect the `Month` column.

<div class="alert alert-info">**unique()** is a very useful function that provides a list of all the unique values of a dataframe column.</div> 

In pandas, we can refer to specific columns of a dataframe using the name of the column (e.g. `df['Month']`). We can see all the unique values of the `Month` column with the following command:

In [102]:
df['Month'].unique()

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., nan])

This almost looks like what we expect, except for the `nan` at the end of our list. It looks like some of the rows in our `.csv` file are missing values in the `Month` column, and pandas has replaced them with `nan`, which is short for "not a number" and the placeholder for empty data in the `pandas` library. We can see how many `nan` values are in a column

In [103]:
df[df['Month'].isna()]

,Rainfall (mm),Date,Year,Month,Day
12053,0.0,12/31/04,2004,NaN,NaN


The pandas dataframe object has a ton of helpful functions. We've already seen **info()**, and **head()**, and **describe()** which are used to provide summaries of the data, as well as **unique()**. A quick `dir(df)` reveals some of what might else be possible.

<div class="alert alert-info">Use the **dir()** command to inspect any object in python and see what methods and attributes it might have</div>

In [104]:
print(dir(df))

['Date', 'Day', 'Month', 'T', 'Year', '_AXIS_ALIASES', '_AXIS_IALIASES', '_AXIS_LEN', '_AXIS_NAMES', '_AXIS_NUMBERS', '_AXIS_ORDERS', '_AXIS_REVERSED', '_AXIS_SLICEMAP', '__abs__', '__add__', '__and__', '__array__', '__array_wrap__', '__bool__', '__bytes__', '__class__', '__contains__', '__copy__', '__deepcopy__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__div__', '__doc__', '__eq__', '__finalize__', '__floordiv__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__iadd__', '__iand__', '__ifloordiv__', '__imod__', '__imul__', '__init__', '__init_subclass__', '__invert__', '__ior__', '__ipow__', '__isub__', '__iter__', '__itruediv__', '__ixor__', '__le__', '__len__', '__lt__', '__matmul__', '__mod__', '__module__', '__mul__', '__ne__', '__neg__', '__new__', '__nonzero__', '__or__', '__pos__', '__pow__', '__radd__', '__rand__', '__rdiv__', '__reduce__', '__reduce_ex__', '__repr__', '__rfloordiv__', '__rmatmul__

Ok. That's a _ton_ of functions. We won't possibly cover all of them, and you could use `pandas` for a very long time before you use them all. Let's look at some of the most important.

### Groupby - quick aggregation

The most common - but not always the most helpful - measure of rainfall is the annual total. Let's see what years we have available to investigate annual rainfall.

In [105]:
df['Year'].unique()

array([1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982,
       1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993,
       1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004,
       2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015,
       2016, 2017])

It looks like we have almost four decades of rainfall data. The first thing we might want to do with this data is look at the yearly rainfall totals. 

Tucked away in the list returned by the **dir()** command is the **groupby()** function, which is one of the most magical aspects of working with dataframes.

In [106]:
group = df.groupby(["Year"])
group.describe()

Day                                                   Month            \
      count       mean       std  min  25%   50%   75%   max  count      mean   
Year                                                                            
1972  366.0  15.756831  8.823592  1.0  8.0  16.0  23.0  31.0  366.0  6.513661   
1973  365.0  15.720548  8.808321  1.0  8.0  16.0  23.0  31.0  365.0  6.526027   
1974  365.0  15.720548  8.808321  1.0  8.0  16.0  23.0  31.0  365.0  6.526027   
1975  365.0  15.720548  8.808321  1.0  8.0  16.0  23.0  31.0  365.0  6.526027   
1976  366.0  15.756831  8.823592  1.0  8.0  16.0  23.0  31.0  366.0  6.513661   
1977  365.0  15.720548  8.808321  1.0  8.0  16.0  23.0  31.0  365.0  6.526027   
1978  365.0  15.720548  8.808321  1.0  8.0  16.0  23.0  31.0  365.0  6.526027   
1979  365.0  15.720548  8.808321  1.0  8.0  16.0  23.0  31.0  365.0  6.526027   
1980  366.0  15.756831  8.823592  1.0  8.0  16.0  23.0  31.0  366.0  6.513661   
1981  396.0  15.742424  8.819387  1.0  8.0  16.0  23.0  31.0  396.0  6.093434   
1982  334.0  15.694611  8.795107  1.0  8.0  16.0  23.0  31.0  334.0  7.038922   
1983  365.0  15.720548  8.808321  1.0  8.0  16.0  23.0  31.0  365.0  6.526027   
1984  366.0  15.756831  8.823592  1.0  8.0  16.0  23.0  31.0  366.0  6.513661   
1985  365.0  15.720548  8.808321  1.0  8.0  16.0  23.0  31.0  365.0  6.526027   
1986  365.0  15.720548  8.808321  1.0  8.0  16.0  23.0  31.0  365.0  6.526027   
1987  366.0  15.680328  8.829837  1.0  8.0  16.0  23.0  31.0  366.0  6.510929   
1988  365.0  15.797260  8.801691  1.0  8.0  16.0  23.0  31.0  365.0  6.528767   
1989  365.0  15.720548  8.808321  1.0  8.0  16.0  23.0  31.0  365.0  6.526027   
1990  365.0  15.720548  8.808321  1.0  8.0  16.0  23.0  31.0  365.0  6.526027   
1991  365.0  15.720548  8.808321  1.0  8.0  16.0  23.0  31.0  365.0  6.526027   
1992  367.0  15.719346  8.840742  1.0  8.0  16.0  23.0  31.0  367.0  6.498638   
1993  364.0  15.758242  8.790918  1.0  8.0  16.0  23.0  31.0  364.0  6.541209   
1994  365.0  15.720548  8.808321  1.0  8.0  16.0  23.0  31.0  365.0  6.526027   
1995  353.0  15.388102  8.743801  1.0  8.0  15.0  23.0  31.0  353.0  6.339943   
1996  378.0  16.066138  8.870322  1.0  8.0  16.0  24.0  31.0  378.0  6.687831   
1997  365.0  15.720548  8.808321  1.0  8.0  16.0  23.0  31.0  365.0  6.526027   
1998  365.0  15.720548  8.808321  1.0  8.0  16.0  23.0  31.0  365.0  6.526027   
1999  365.0  15.720548  8.808321  1.0  8.0  16.0  23.0  31.0  365.0  6.526027   
2000  366.0  15.756831  8.823592  1.0  8.0  16.0  23.0  31.0  366.0  6.513661   
2001  365.0  15.720548  8.808321  1.0  8.0  16.0  23.0  31.0  365.0  6.526027   
2002  365.0  15.720548  8.808321  1.0  8.0  16.0  23.0  31.0  365.0  6.526027   
2003  365.0  15.720548  8.808321  1.0  8.0  16.0  23.0  31.0  365.0  6.526027   
2004  365.0  15.720548  8.808321  1.0  8.0  16.0  23.0  31.0  365.0  6.526027   
2005  365.0  15.720548  8.808321  1.0  8.0  16.0  23.0  31.0  365.0  6.526027   
2006  365.0  15.720548  8.808321  1.0  8.0  16.0  23.0  31.0  365.0  6.526027   
2007  365.0  15.720548  8.808321  1.0  8.0  16.0  23.0  31.0  365.0  6.526027   
2008  366.0  15.756831  8.823592  1.0  8.0  16.0  23.0  31.0  366.0  6.513661   
2009  365.0  15.720548  8.808321  1.0  8.0  16.0  23.0  31.0  365.0  6.526027   
2010  365.0  15.720548  8.808321  1.0  8.0  16.0  23.0  31.0  365.0  6.526027   
2011  365.0  15.720548  8.808321  1.0  8.0  16.0  23.0  31.0  365.0  6.526027   
2012  366.0  15.756831  8.823592  1.0  8.0  16.0  23.0  31.0  366.0  6.513661   
2013  365.0  15.720548  8.808321  1.0  8.0  16.0  23.0  31.0  365.0  6.526027   
2014  365.0  15.720548  8.808321  1.0  8.0  16.0  23.0  31.0  365.0  6.526027   
2015  365.0  15.720548  8.808321  1.0  8.0  16.0  23.0  31.0  365.0  6.526027   
2016  366.0  15.756831  8.823592  1.0  8.0  16.0  23.0  31.0  366.0  6.513661   
2017  165.0  14.933333  8.777836  1.0  7.0  14.0  22.0  31.0  165.0  3.266667   

        ...                 Rainfall (

The result of **groupby()** is normally a `pandas` object called a `group`. This `group` object contains aggregated information on each column in our dataframe, organized according to the values of the aggregation column or columns (in this case, we chose to aggregate our data by `Year`). 

The result of the **groupby()** command can contain a lot of weird results. For example, we see that the `mean` value of the `Day` column is `15.720548`, except for some years, when it is `15.756831`.

<div class="alert alert-warning">Pandas functions, like **groupby()** will often _work_, but it is up to you to provide context and make sure the calculations have _meaning_.</div>

Assuming we're not interested in summary statistics of the days in a year, we can force pandas to focus on the columns we are interested in when using the **groupby()** command.

In [107]:
yearly_rainfall = df.groupby(["Year"])['Rainfall (mm)']
yearly_rainfall.describe()

,count,mean,std,min,25%,50%,75%,max
Year,,,,,,,,
1972,366.0,1.861279,6.588440,0.0,0.0,0.0,0.0,81.280
1973,365.0,-9999.000000,0.000000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.000
1974,365.0,1.038268,3.955957,0.0,0.0,0.0,0.0,45.720
1975,365.0,2.168395,6.392157,0.0,0.0,0.0,0.0,53.340
1976,366.0,0.992404,3.335284,0.0,0.0,0.0,0.0,27.432
1977,365.0,2.320795,6.757021,0.0,0.0,0.0,0.0,55.880
1978,365.0,2.146822,7.078181,0.0,0.0,0.0,0.0,69.850
1979,365.0,1.472504,6.293745,0.0,0.0,0.0,0.0,66.040
1980,366.0,1.190191,5.353942,0.0,0.0,0.0,0.0,64.770


This is much better. Now we have aggregated our data of `Rainall (mm)` by `Year`, and `pandas` has helpfully provided summary statistics of the rainfall in each year. Unfortunately, we see that the most interesting summary, the `sum` of yearly rainfall is not provided. And we also see that the minimum value in some years is `-9999`, which probably means that there are some missing data. We need to filter out these missing data so they don't affect our analyses.

Now we need to introduce one more important component of our `pandas` workflow...

### Method chaining

[Method chaining](https://en.wikipedia.org/wiki/Method_chaining) is a means of concatenating functions in order to quickly complete a series of data transformations. In pandas, we often use method chaining in aggregation processes to perfrom calculations on groups or selections of data. Methods are appended using `.` notation to the end of a command. Any code that is expressed using method chaining could be written using a series of commands. For example,

```python
a = [10, 2, 5, 5, 7, 8, 23, 1]
b = a.sort()
```

is the same as this statement that uses method chaining

```python
b = [10, 2, 5, 5, 7, 8, 23, 1].sort()
```

<div class="alert alert-info">Method chaining is a concept common in many object-oriented programming languages, particularly `javascript`. While not widely-used in `Python`, it is often found in `pandas` applications.</div>

Let's use method chaining combined with the **groupby()** function to get the sum of every year's rainfall. We do this by _chaining_ the result of the `df.groupby()` command to the `sum()` command.

In [108]:
yearly_rainfall_total = df.groupby(["Year"])['Rainfall (mm)'].sum()
yearly_rainfall_total.head()

Year
1972        681.228
1973   -3649635.000
1974        378.968
1975        791.464
1976        363.220
Name: Rainfall (mm), dtype: float64

😀 🌈 While the specifics of panda's API (application programming interface) may still feel very vague and arbitrary, this should be your first _Ah-Ha!_ moment. We just iterated over 40 years of rainfall data, and quickly determined the annual rainfall totals with a single line of code. 

And we can get much more ambitious... what about monthly rainfall?

In [109]:
monthly_rainfall = df.groupby(["Year", "Month"])["Rainfall (mm)"].sum()
monthly_rainfall.head()

Year  Month
1972  1.0        0.000
      2.0       33.020
      3.0        0.000
      4.0       14.732
      5.0      233.680
Name: Rainfall (mm), dtype: float64

<div class="alert alert-info">The result of **groupby()** is normally a `pandas` structure called a `group`. However, by using the `sum()` command, we are asking `pandas` to take the output 

<div class="alert alert-success">
   ✏️ __DIY Coding__: What was the **maximum daily rainfall** in each month over the period of record?
</div>


In [110]:
# This cell intentionally left blank.

### Visualizing Data

The plotting universe in python is very diverse. The most common library for plotting data in python is [`matplotlib`](https://en.wikipedia.org/wiki/Matplotlib), which provides a plotting interface very similar to `MATLAB`'s. 

There are additional libraries built on top of `matplotlib`, such as [`seaaborn`](https://seaborn.pydata.org), which is designed for statistical analyses, and [`holoviews`](http://holoviews.org/reference/), which is deisgned for more semantic data visualization. There are other libraries desgined based on `R`'s _Grammar of Graphics_ system, such as  [`ggplot`](http://ggplot.yhathq.com), which is an interface ported from `R`'s `ggplot2` library, and [`Bokeh`](https://bokeh.pydata.org/en/latest/). 

In addition, there are visualization packages that add interactivity and web-enabled graphics to your plots, such as [`plotly`](https://plot.ly). 

All of these are free, except for plotly, which requires a paid student license to generate private plots. You can find a recent (May, 2018) overview of each of these frameworks, plus a few more, at [fusioncharts](https://www.fusioncharts.com/blog/best-python-data-visualization-libraries/).

For our first example of plotting annual and monthly rainfall data, we will use `Bokeh`. It has a very intuitive interface, and also happens to do bar charts very well.

First, let's import all the necessary objects from the `bokeh` library. We import the `figure` object and the `ColumnDataSource` object. The `figure` object will be used to generate a figure, while the `ColumnDataSource` is an object unique to `Bokeh`, which is where the data to be used in the plot will be stored. Luckily, `bokeh` works fine with pandas `DataFrames`, and can even be used with dataframe `groups` created by the **groupby()** function, so we are all set.

In [111]:
from bokeh.plotting import figure
from bokeh.io import output_notebook

<div class="alert alert-info">The **output_notebook()** function tells `bokeh` to render plots into the jupyter notebook.</alert>

1. Save the results of the **groupby()** function to a new variable, we'll call `group`.

In [112]:
group = df.groupby(["Year"])["Rainfall (mm)"].agg('sum')
group.describe()

count    4.600000e+01
mean    -1.844396e+05
std      7.679435e+05
min     -3.649635e+06
25%      4.147820e+02
50%      4.737100e+02
75%      6.998335e+02
max      1.182370e+03
Name: Rainfall (mm), dtype: float64

2. Use the `bokeh` **ColumnDataSource()** function to generate an object for plotting.

In [113]:
source = ColumnDataSource(group)
p = figure()
p.circle(x='x_values', y='y_values', source=source)

show(p)

NameError: name 'ColumnDataSource' is not defined